# Modelo 1

## XGBoost

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

from xgboost import XGBClassifier

In [2]:
dfTransaction = pd.read_csv("../Dataset/ML/train_transaction.csv")
dfIdentity = pd.read_csv("../Dataset/ML/train_identity.csv")

# Uso left ya que pueden haber transacciones sin identidades pero no puede suceder lo contrario
dfMerge = dfTransaction.merge(dfIdentity, on = "TransactionID", how = "left")

del dfTransaction
del dfIdentity
dfMerge.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


### Split del dataset

In [3]:
train_size = int(len(dfMerge) * 0.75)

X_train = dfMerge.iloc[:train_size, 1:] # Saco el Transaction ID
X_val = dfMerge.iloc[train_size:, 1:] # Saco el Transaction ID
y_train = dfMerge["isFraud"].iloc[:train_size]
y_val = dfMerge["isFraud"].iloc[train_size:]

In [4]:
del dfMerge

In [5]:
X_train.tail()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
442900,0,11246397,107.950,W,7508,321.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442901,0,11246476,35.658,C,15885,545.0,185.0,visa,138.0,debit,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
442902,0,11246502,30.950,W,9992,455.0,150.0,mastercard,126.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442903,0,11246585,57.950,W,9485,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442904,0,11246605,57.950,W,1919,321.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
X_val.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
442905,0,11246665,30.95,W,8528,215.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442906,0,11246704,53.95,W,7919,194.0,150.0,mastercard,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442907,0,11246761,117.00,W,15497,490.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442908,0,11246761,29.00,W,7826,481.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
442909,0,11247072,200.00,R,2528,399.0,150.0,american express,137.0,credit,...,mobile safari generic,32.0,2732x2048,match_status:1,T,F,F,F,mobile,iOS Device


### Encoding

#### CountVectorizer

In [7]:
columnas_vectorizer = []

In [8]:
# Utilizo CountVectorizer para el campo id_31
X_train["id_31"].fillna("NaN", inplace = True)
X_val["id_31"].fillna("NaN", inplace = True)
vectorizer_browsers = CountVectorizer(max_features = 15, dtype = bool)

browsers_matrix = vectorizer_browsers.fit_transform(X_train["id_31"].values).todense()
browsers_matrix_val = vectorizer_browsers.transform(X_val["id_31"].values).todense()

vectorizer_browsers.vocabulary_

{'nan': 13,
 'mobile': 12,
 'safari': 14,
 '11': 0,
 'chrome': 6,
 '62': 1,
 'for': 9,
 'android': 5,
 'generic': 10,
 'firefox': 8,
 'ie': 11,
 'desktop': 7,
 '63': 2,
 '64': 3,
 '65': 4}

In [9]:
# Me quedo con los nombres de los navegadores
browsers = ["chrome", "safari", "firefox", "ie", "nan"]
for browser in browsers:
    columnas_vectorizer += ["browser_{}".format(browser)]
    indice = vectorizer_browsers.vocabulary_[browser]
    
    X_train["browser_{}".format(browser)] = list(np.asarray(browsers_matrix)[:,indice])
    X_val["browser_{}".format(browser)] = list(np.asarray(browsers_matrix_val)[:,indice])
X_train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_36,id_37,id_38,DeviceType,DeviceInfo,browser_chrome,browser_safari,browser_firefox,browser_ie,browser_nan
0,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,True
1,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,True
2,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,True
3,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,False,False,False,False,True
4,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,False,False,False,False,False


In [10]:
del browsers_matrix
del browsers_matrix_val

In [11]:
# Utilizo CountVectorizer para el campo id_30
X_train["id_30"].fillna("NaN", inplace = True)
X_val["id_30"].fillna("NaN", inplace = True)
vectorizer_sos = CountVectorizer(max_features = 15, dtype = bool)

sos_matrix = vectorizer_sos.fit_transform(X_train["id_30"].values).todense()
sos_matrix_val = vectorizer_sos.transform(X_val["id_30"].values).todense()

vectorizer_sos.vocabulary_

{'nan': 12,
 'android': 8,
 'ios': 9,
 '11': 7,
 'mac': 11,
 'os': 13,
 '10_11_6': 2,
 'windows': 14,
 '10': 0,
 'linux': 10,
 '10_12_6': 3,
 '10_13_1': 4,
 '10_10_5': 1,
 '10_13_2': 5,
 '10_13_3': 6}

In [12]:
# Me quedo con los nombres de los SOs
sos = ["android", "ios", "mac", "linux", "windows", "nan"]
for so in sos:
    columnas_vectorizer += ["SO_{}".format(so)]
    indice = vectorizer_sos.vocabulary_[so]
    
    X_train["SO_{}".format(so)] = list(np.asarray(sos_matrix)[:,indice])
    X_val["SO_{}".format(so)] = list(np.asarray(sos_matrix_val)[:,indice])
X_train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,browser_safari,browser_firefox,browser_ie,browser_nan,SO_android,SO_ios,SO_mac,SO_linux,SO_windows,SO_nan
0,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,False,False,False,True,False,False,False,False,False,True
1,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,False,False,False,True,False,False,False,False,False,True
2,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,False,False,False,True,False,False,False,False,False,True
3,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,False,False,False,True,False,False,False,False,False,True
4,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,False,False,False,False,True,False,False,False,False,False


In [13]:
del sos_matrix
del sos_matrix_val

#### One Hot Encoding

In [14]:
# Hago OHE de las siguiente columnas
oh_columns = ["ProductCD", "card4", "card6", "DeviceType"]
ohe = OneHotEncoder(sparse = False, dtype = bool, handle_unknown = "ignore")

ohe_matrix = ohe.fit_transform(X_train[oh_columns])
ohe_matrix_val = ohe.transform(X_val[oh_columns])

# Elimino estas 2 categorias que solo aparecen 50 veces en todo el dataset
X_train[list(ohe.get_feature_names_out(oh_columns))] = ohe_matrix
X_train.drop(["card6_debit or credit", "card6_charge card"], axis = 1, inplace = True)

X_val[list(ohe.get_feature_names_out(oh_columns))] = ohe_matrix_val
X_val.drop(["card6_debit or credit", "card6_charge card"], axis = 1, inplace = True)

In [15]:
del ohe_matrix
del ohe_matrix_val

In [16]:
columnas_ohe = list(ohe.get_feature_names_out(oh_columns))
columnas_ohe.remove("card6_debit or credit")
columnas_ohe.remove("card6_charge card")

In [17]:
X_train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,card4_discover,card4_mastercard,card4_visa,card4_nan,card6_credit,card6_debit,card6_nan,DeviceType_desktop,DeviceType_mobile,DeviceType_nan
0,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,True,False,False,False,True,False,False,False,False,True
1,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,False,True,False,False,True,False,False,False,False,True
2,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,False,False,True,False,False,True,False,False,False,True
3,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,False,True,False,False,False,True,False,False,False,True
4,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,False,True,False,False,True,False,False,False,True,False


In [18]:
# Obtengo la cantidad de pixeles de la columna id_33
def parse_resolucion(x):
    if pd.isna(x):
        return np.NaN
    pixeles = x.split("x")
    return int(pixeles[0]) * int(pixeles[1])

X_train["resolucion"] = X_train["id_33"].apply(lambda x: parse_resolucion(x))
X_val["resolucion"] = X_val["id_33"].apply(lambda x: parse_resolucion(x))

In [19]:
X_train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,card4_mastercard,card4_visa,card4_nan,card6_credit,card6_debit,card6_nan,DeviceType_desktop,DeviceType_mobile,DeviceType_nan,resolucion
0,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,...,False,False,False,True,False,False,False,False,True,NaN
1,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,...,True,False,False,True,False,False,False,False,True,NaN
2,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,...,False,True,False,False,True,False,False,False,True,NaN
3,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,...,True,False,False,False,True,False,False,False,True,NaN
4,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,...,True,False,False,True,False,False,False,True,False,2397600.0


In [20]:
X_train.drop(oh_columns + ["id_30", "id_31", "id_33"], axis = 1, inplace = True)
X_val.drop(oh_columns + ["id_30", "id_31", "id_33"], axis = 1, inplace = True)

#### Mean Encoding

In [21]:
columnas_mean = ["card1", "card2", "card3", "card5", "addr1", "addr2",
                "P_emaildomain", "R_emaildomain", "M1", "M2", "M3", "M4", "M5", "M6", "M7", "M8", "M9",
                "DeviceInfo", 'id_12', 'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18',
                'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28',
                'id_29', 'id_32', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']
means = []
for columna in columnas_mean:
    mean_encoded = X_train.groupby(columna)['isFraud'].mean().to_dict()
    means += [mean_encoded]
    
    X_train[columna] = X_train[columna].map(mean_encoded)
    X_val[columna] = X_val[columna].map(mean_encoded)

In [22]:
# Tratamiento de nulos
for columna in X_train.columns:
    X_train[columna].fillna(X_train[columna].mean(), inplace = True)
    X_val[columna].fillna(X_val[columna].mean(), inplace = True)

In [23]:
X_train.drop("isFraud", axis = 1, inplace = True)
X_val.drop("isFraud", axis = 1, inplace = True)

In [25]:
columnas_modelo = columnas_ohe + columnas_mean + columnas_vectorizer + ["resolucion",
                                                                        "TransactionDT", "TransactionAmt", "D1", "D2", "D3", "D4",
                                                                        "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12", "D13",
                                                                        "D14", "dist1", "dist2"]

In [26]:
X_train = X_train[columnas_modelo]
X_val = X_val[columnas_modelo]

### XGBoost con RandomSearch

In [24]:
model_params = {
    'max_depth': range(3, 11),
    'min_child_weight': [0.3, 0.5, 1.0, 1.5, 3.0],
    'gamma': [0, 0.25, 0.5, 1.0],
    'subsample': np.arange(0.5, 1.01, 0.1),
    'colsample_bytree': np.arange(0.5, 1.01, 0.1),
    'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
    'reg_alpha': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2]
}

In [28]:
xgboost = XGBClassifier(use_label_encoder = False, booster = "gbtree", n_estimators = 100, n_jobs = 1)
classifier = RandomizedSearchCV(xgboost, model_params, n_iter = 20,
                            n_jobs = 1, verbose = 100, cv = 3,
                            scoring = 'roc_auc', random_state = 1, error_score = "raise")

##### Si hay poroblemas de memoria, se puede guardar la información del Notebook, reiniciarlo y volver a cargarlo. Esto ayuda a liberar memoria _leakeada_ por Jupyter

In [30]:
import dill
dill.dump_session('dump_xgboost.db')

> Reiniciar kernel

In [1]:
import dill
dill.load_session('dump_xgboost.db')

In [2]:
classifier.fit(X_train.values, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV 1/3; 1/20] START colsample_bytree=0.5, gamma=0.5, learning_rate=0.2, max_depth=9, min_child_weight=1.0, reg_alpha=50.0, reg_lambda=10.0, subsample=0.7999999999999999
[17:52:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV 1/3; 1/20] END colsample_bytree=0.5, gamma=0.5, learning_rate=0.2, max_depth=9, min_child_weight=1.0, reg_alpha=50.0, reg_lambda=10.0, subsample=0.7999999999999999;, score=0.896 total time= 1.6min
[CV 2/3; 1/20] START colsample_bytree=0.5, gamma=0.5, learning_rate=0.2, max_depth=9, min_child_weight=1.0, reg_alpha=50.0, reg_lambda=10.0, subsample=0.7999999999999999
[17:53:58] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was

RandomizedSearchCV(cv=3, error_score='raise',
                   estimator=XGBClassifier(base_score=None, booster='gbtree',
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=...
                   param_distributions={'colsample_bytree': array([0.5, 0.6, 0.7, 0.8, 0.9, 1. ]),
                                        'gamma': [0, 0.25, 0.5, 1.0],
                                        'learning_rate'

In [3]:
print("Best train score: {}".format(classifier.best_score_))
print("Best params: ")
for param_name in sorted(classifier.best_params_.keys()):
    print('%s: %r' % (param_name, classifier.best_params_[param_name]))

Best train score: 0.8781506587374238
Best params: 
colsample_bytree: 0.7
gamma: 1.0
learning_rate: 0.05
max_depth: 10
min_child_weight: 0.5
reg_alpha: 10.0
reg_lambda: 100.0
subsample: 0.6


In [5]:
predictions = classifier.predict_proba(X_val.values)
print("Validation score: %r" % roc_auc_score(y_val, [pred[1] for pred in predictions]))

Validation score: 0.8218925423889694


#### Predicción del set de Test para la competencia

In [6]:
del X_train
del y_train
del X_val
del y_val

In [23]:
dfTestTransaction = pd.read_csv("../Dataset/ML/test_transaction.csv")
dfTestIdentity = pd.read_csv("../Dataset/ML/test_identity.csv")
dfTest = dfTestTransaction.merge(dfTestIdentity, on = "TransactionID", how = "left")

del dfTestTransaction
del dfTestIdentity

In [24]:
dfTest.columns = [columna.replace("id-", "id_") for columna in dfTest.columns]

In [25]:
dfTest["id_31"].fillna("NaN", inplace = True)
browsers_matrix = vectorizer_browsers.transform(dfTest["id_31"].values).todense()

for browser in browsers:
    indice = vectorizer_browsers.vocabulary_[browser]
    dfTest["browser_{}".format(browser)] = list(np.asarray(browsers_matrix)[:,indice])

In [26]:
del browsers_matrix

In [27]:
dfTest["id_30"].fillna("NaN", inplace = True)
sos_matrix = vectorizer_sos.transform(dfTest["id_30"].values).todense()

for so in sos:
    indice = vectorizer_sos.vocabulary_[so]
    dfTest["SO_{}".format(so)] = list(np.asarray(sos_matrix)[:,indice])

In [28]:
del sos_matrix

In [29]:
ohe_matrix = ohe.transform(dfTest[oh_columns])

dfTest[list(ohe.get_feature_names_out(oh_columns))] = ohe_matrix
dfTest.drop(["card6_debit or credit", "card6_charge card"], axis = 1, inplace = True)

In [30]:
del ohe_matrix

In [31]:
dfTest["resolucion"] = dfTest["id_33"].apply(lambda x: parse_resolucion(x))

In [32]:
dfTest.drop(oh_columns + ["id_30", "id_31", "id_33"], axis = 1, inplace = True)

In [33]:
for i, columna in enumerate(columnas_mean):
    mean_encoded = means[i]
    dfTest[columna] =  dfTest[columna].map(mean_encoded)

In [34]:
for columna in dfTest.columns:
    dfTest[columna].fillna(dfTest[columna].mean(), inplace = True)

In [35]:
# Me deshago de las columnas que no necesito para ahorrar memoria
# Me quedo con TransactionID para el submission, pero no lo uso para predecir
dfTest = dfTest[["TransactionID"] + columnas_modelo]

In [ ]:
predictions = classifier.predict_proba(dfTest[columnas_modelo].values)
dfTest["isFraud"] = [pred[1] for pred in predictions]
dfTest.head()

,TransactionID,ProductCD_C,ProductCD_H,ProductCD_R,ProductCD_S,ProductCD_W,card4_american express,card4_discover,card4_mastercard,card4_visa,...,D8,D9,D10,D11,D12,D13,D14,dist1,dist2,isFraud
0,3663549,False,False,False,False,True,False,False,False,True,...,160.834483,0.553981,418.0,203.0,77.404179,18.225961,58.163186,1.0,237.175047,0.008856
1,3663550,False,False,False,False,True,False,False,False,True,...,160.834483,0.553981,231.0,634.0,77.404179,18.225961,58.163186,4.0,237.175047,0.036266
2,3663551,False,False,False,False,True,False,False,False,True,...,160.834483,0.553981,136.0,136.0,77.404179,18.225961,58.163186,2635.0,237.175047,0.009803
3,3663552,False,False,False,False,True,False,False,False,True,...,160.834483,0.553981,242.0,242.0,77.404179,18.225961,58.163186,17.0,237.175047,0.009757
4,3663553,False,False,False,False,True,False,False,True,False,...,160.834483,0.553981,22.0,22.0,77.404179,18.225961,58.163186,6.0,237.175047,0.027894


In [ ]:
dfTest[["TransactionID", "isFraud"]].to_csv("xgboost_submission.csv", index = False)

El score obtenido en Kaggle es de `0.832691`. El csv con las predicciones se encuentra en https://github.com/ManuelBilbao/75.06-OrgaDeDatos-TPs/tree/main/ML/xgboost_submission.csv